In [1]:
import camelot
import pandas as pd

In [2]:
tables = camelot.read_pdf("VAT_2025.12_signed.pdf", pages='all')

In [14]:
stmnt_name = tables[0].df[0][0].replace('\n', "")
"ՄԻԱՍՆԱԿԱՆ ՀԱՇՎԱՐԿԱՎԵԼԱՑՎԱԾ ԱՐԺԵՔԻ ՀԱՐԿԻ ԵՎԱԿՑԻԶԱՅԻՆ ՀԱՐԿԻ" not in stmnt_name


False

In [21]:
total_amount = 0
for i in range(1, 13):
    tables = camelot.read_pdf(f'files/VAT_2024.{i}_signed.pdf',
                              suppress_stdout=True,
                              pages="all")
    df = tables[1].df
    res = int(df[1][21].replace(",", ""))
    total_amount += res

    


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, def

In [3]:
total_amount

131349911

In [3]:
tables = camelot.read_pdf('files/VAT_2024.1_signed.pdf', suppress_stdout=True)
type(tables)

CropBox missing from /Page, defaulting to MediaBox


camelot.core.TableList

In [29]:
tables[0].parsing_report

{'accuracy': 100.0, 'whitespace': 0.0, 'order': 1, 'page': 1}

In [43]:
df = tables[1].df
type(df)

pandas.core.frame.DataFrame

In [37]:
res = df[1][21]
res = res.replace(",", "")
int(res)

11827993

In [ ]:
tables = camelot.read_pdf('files/VAT_2024.1_signed.pdf', suppress_stdout=True,
                          pages="all")

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


In [110]:
tables.n
type(tables)

camelot.core.TableList

In [111]:

def get_company(tables: camelot.core.TableList) -> dict:
  df: pd.DataFrame = tables[0].df
  company: str = df[1][3]
  tin: str = df[0][2].replace('\n', "")
  res: dict = {"Company": company, "TIN": tin}
  return res

def get_period(tables: camelot.core.TableList) -> dict:
  df: pd.DataFrame = tables[0].df
  period: str = df[1][5].replace('\n', "")
  year: int = int(period[4:8])
  month: int = int(period[12:])
  res: dict = {"Year": year, "Month": month}
  return res

data = get_period(tables)
data.update(get_company(tables))
data

{'Year': 2023, 'Month': 1, 'Company': '«ԳՐԻՆ ՍԼՈՊ»', 'TIN': '00169872'}

In [112]:

def get_credit_data(tables: camelot.core.TableList) -> dict:
  df: pd.DataFrame = tables[1].df
  revenue = df[1][21].replace(",", "")
  if not revenue:
    revenue = 0
  revenue = int(revenue)

  vat_credit = df[2][21].replace(",", "")
  if not vat_credit:
    vat_credit = 0
  vat_credit = int(vat_credit)

  return {"Revenue": revenue, "VAT_credit": vat_credit}

def get_debit_data(tables: camelot.core.TableList) -> dict:
  df: pd.DataFrame = tables[1].df
  import_amount = df[1][23].replace(",", "")
  import_vat = df[2][23].replace(",", "")
  inner_purchase_amount = df[1][24].replace(",", "")
  inner_purchase_vat = df[2][24].replace(",", "")
  if not import_amount:
    import_amount = 0
  if not import_vat:
    import_vat = 0
  if not inner_purchase_amount:
    inner_purchase_amount = 0
  if not inner_purchase_vat:
    inner_purchase_vat = 0
  import_amount = int(import_amount)
  import_vat = int(import_vat)
  inner_purchase_amount = int(inner_purchase_amount)
  inner_purchase_vat = int(inner_purchase_vat)
  
  return {
    "Import amount": import_amount, "Import VAT": import_vat,
    "Inner purchase amount": inner_purchase_amount,
    "Inner purchase VAT": inner_purchase_vat
    }

data.update(get_credit_data(tables))
data.update(get_debit_data(tables))
data


{'Year': 2023,
 'Month': 1,
 'Company': '«ԳՐԻՆ ՍԼՈՊ»',
 'TIN': '00169872',
 'Revenue': 7508223,
 'VAT_credit': 0,
 'Import amount': 5324772,
 'Import VAT': 0,
 'Inner purchase amount': 4766530,
 'Inner purchase VAT': 727806}

In [114]:

def get_vat_total(tables: camelot.core.TableList) -> dict:
  df: pd.DataFrame = tables[2].df
  vat_total_credit = df[1][5].replace(",", "")
  vat_total_debit = df[2][5].replace(",", "")
  if not vat_total_credit:
    vat_total_credit = 0
  if not vat_total_debit:
    vat_total_debit = 0
  vat_total_credit = int(vat_total_credit)
  vat_total_debit = int(vat_total_debit)
  if vat_total_credit == 0 and vat_total_debit == 0:
    return {"VAT total": 0}
  if vat_total_credit > 0:
    return {"VAT total": vat_total_credit}
  if vat_total_debit > 0:
    return {"VAT total": vat_total_debit * -1}

data.update(get_vat_total(tables))
data

{'Year': 2023,
 'Month': 1,
 'Company': '«ԳՐԻՆ ՍԼՈՊ»',
 'TIN': '00169872',
 'Revenue': 7508223,
 'VAT_credit': 0,
 'Import amount': 5324772,
 'Import VAT': 0,
 'Inner purchase amount': 4766530,
 'Inner purchase VAT': 727806,
 'VAT total': -727806}

In [122]:
data = []
for i in range(1, 13):
  tables = camelot.read_pdf(f'files/VAT_2024.{i}_signed.pdf', suppress_stdout=True,
                          pages="all")
  stmt = {}
  stmt.update(get_company(tables))
  stmt.update(get_period(tables))
  stmt.update(get_credit_data(tables))
  stmt.update(get_debit_data(tables))
  stmt.update(get_vat_total(tables))
  data.append(stmt)

data
  

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, def

[{'Company': '«ԳՐԻՆ ՍԼՈՊ»',
  'TIN': '00169872',
  'Year': 2023,
  'Month': 1,
  'Revenue': 7508223,
  'VAT_credit': 0,
  'Import amount': 5324772,
  'Import VAT': 0,
  'Inner purchase amount': 4766530,
  'Inner purchase VAT': 727806,
  'VAT total': -727806},
 {'Company': '«ԳՐԻՆ ՍԼՈՊ»',
  'TIN': '00169872',
  'Year': 2023,
  'Month': 2,
  'Revenue': 20162543,
  'VAT_credit': 0,
  'Import amount': 5041057,
  'Import VAT': 213605,
  'Inner purchase amount': 6614720,
  'Inner purchase VAT': 1247944,
  'VAT total': -1461549},
 {'Company': '«ԳՐԻՆ ՍԼՈՊ»',
  'TIN': '00169872',
  'Year': 2023,
  'Month': 3,
  'Revenue': 4886966,
  'VAT_credit': 0,
  'Import amount': 5221445,
  'Import VAT': 253296,
  'Inner purchase amount': 2280666,
  'Inner purchase VAT': 330777,
  'VAT total': -584073},
 {'Company': '«ԳՐԻՆ ՍԼՈՊ»',
  'TIN': '00169872',
  'Year': 2023,
  'Month': 4,
  'Revenue': 9323274,
  'VAT_credit': 0,
  'Import amount': 2396886,
  'Import VAT': 326699,
  'Inner purchase amount': 6722895,

In [141]:
df = pd.DataFrame(data)
df["Debit_t/Revenue"] = abs(round(df["VAT total"] / df["Revenue"] * 100, 2))
df[["Month", "Revenue", "VAT total", "Debit_t/Revenue"]]

,Month,Revenue,VAT total,Debit_t/Revenue
0,1,7508223,-727806,9.69
1,2,20162543,-1461549,7.25
2,3,4886966,-584073,11.95
3,4,9323274,-1633378,17.52
4,5,3630010,-355490,9.79
5,6,8575740,-1369099,15.96
6,7,33817936,-4743606,14.03
7,8,26438614,-264232,1.00
8,9,0,0,NaN
9,10,15408496,-2795482,18.14


### Get business code and percenage

In [ ]:
df = tables[4].df
def get_business_code_and_p(df: pd.DataFrame) -> dict:
  business_type = df[0][6].replace("\n", "")
  business_code = business_type[:6]
  percent = business_type[6:]
  business_code, percent

('G46769', '100')